Bayesian interpretation of medical tests
-----------------------------------------

An example that demonstrates a simple hierarchical model.

Copyright 2016 Allen Downey

MIT License: http://opensource.org/licenses/MIT

In [1]:
from __future__ import print_function, division

import thinkplot
from thinkbayes2 import Pmf, Suite
from thinkbayes2 import MakeGeometricPmf
from thinkbayes2 import MakeMixture

import numpy as np
from fractions import Fraction

%matplotlib inline

### A dice problem

Before we get started, let's warm up with a dice problem.  It will introduce the computational framework I'm using, and provide some background we'll need for the medical test problem.

Suppose I have a six-sided die that is red on 2 sides and blue on 4 sides, and another die that's the other way around, red on 4 sides and blue on two.

I choose a die at random and roll it, and I tell you it came up red.  What is the probability that I rolled the second die (red on 4 sides)?

To answer this question, I'll create `Pmf` objects to represent the two dice:

In [2]:
d1 = Pmf({'Red':Fraction(2), 'Blue':Fraction(4)}, label='d1 (bluish) ')
d1.Print()

Blue 2/3
Red 1/3


In [3]:
d2 = Pmf({'Red':Fraction(4), 'Blue':Fraction(2)}, label='d2 (reddish)')
d2.Print()

Blue 1/3
Red 2/3


And I'll make another `Pmf` to represent the random choice of one die or the other.

In [4]:
dice = Pmf({d1:Fraction(1), d2:Fraction(1)})
dice.Print()

d2 (reddish) 1/2
d1 (bluish)  1/2


Now I can use the `Random` method to choose a die and then roll it.

In [5]:
dice.Random().Random()

'Blue'

The following generator simulates the process of repeatedly choosing a die and then rolling it.

In [6]:
def rollA(dice):
    while True:
        die = dice.Random()
        roll = die.Random()
        yield roll

We can use this generator to simulate rolls:

In [7]:
iterA = rollA(dice)
for i in range(5):
    print(next(iterA))

Red
Red
Red
Red
Red


In the long run, the proportion of red and blue is 50-50.

In [8]:
Pmf(next(iterA) for i in range(1000))

Pmf({'Red': 0.47800000000000004, 'Blue': 0.522})

We can see that by computing the weighted mixture of the two dice:

In [9]:
MakeMixture(dice).Print()

Blue 1/2
Red 1/2


To answer the original question, I'll create a suite of hypotheses where each hypothesis is represented by a die, and the likelihood of the data under each hypothesis is the probability that the given die yields the given outcome.

In [10]:
class Dice(Suite):
    def Likelihood(self, data, hypo):
        """
        data: 'Red' or 'Blue'
        hypo: a Die object
        """
        return hypo[data]

Now I can create a suite that represents the prior distribution.

In [11]:
prior = Dice({d1:Fraction(1), d2:Fraction(1)})
prior.Print()

d2 (reddish) 1/2
d1 (bluish)  1/2


And update it with the data.

In [12]:
posterior = prior.Copy()
posterior.Update('Red')
posterior.Print()

d2 (reddish) 2/3
d1 (bluish)  1/3


The posterior probabilities for `d1` and `d2` are `1/3` and `2/3`.  That is, the probability that I rolled `d2` is `2/3`.

Intuitively, the posterior probability of `d2` is higher than the prior because the outcome (red) is more likely on `d2` than `d1`.  If we had rolled blue, the probability of `d1` would be higher.

Now suppose I ask you to predict the outcome of the next roll.  Remember that in this scenario, I choose a new die each time, so what you learned on the first roll doesn't help you with the second.  The predictive distribution is a weighted mixture of the dice, using the prior weights:

In [13]:
predictive = MakeMixture(prior)
predictive.Print()

Blue 1/2
Red 1/2


Now consider a different scenario.  Instead of choosing a new die every time, I choose a die once and roll it repeatedly.  Here's a generator that simulates this scenario:

In [14]:
def rollB(dice):
    die = dice.Random()
    while True:
        roll = die.Random()
        yield roll

In the long run, the proportion of red is either `1/3` or `2/3`, not `1/2`:

In [15]:
iterB = rollB(dice)
Pmf(next(iterB) for i in range(1000))

Pmf({'Red': 0.327, 'Blue': 0.673})

After the first roll, the posterior suite is the same as in the previous scenario:

In [16]:
posterior = prior.Copy()
posterior.Update('Red')
posterior.Print()

d2 (reddish) 2/3
d1 (bluish)  1/3


In this scenario, we know we are going to roll the same die each time, so the information we learned from the first roll informs our prediction for the second.

Specifically, now the predictive distribution is based on the posterior, not the prior:

In [17]:
predictive = MakeMixture(posterior)
predictive.Print()

Blue 4/9
Red 5/9


Having seen one red, we are more inclined to belive that I am rolling `d2`, so we are more inclined to predict that I will roll red again.

If I do roll red again, we can update the posterior again, using the new data, and make a new prediction:

In [18]:
posterior.Update('Red')
posterior.Print()

d2 (reddish) 4/5
d1 (bluish)  1/5


In [19]:
predictive = MakeMixture(posterior)
predictive.Print()

Blue 2/5
Red 3/5


If we continue this process, we will eventually be confident that we know which die is being rolled:

In [20]:
posterior = prior.Copy()
for i in range(10):
    posterior.Update(next(iterB))
posterior.Print()

d2 (reddish) 1/257
d1 (bluish)  256/257


And the predictive distribution will be close to `1/3` or `2/3`, depending on which die we think it is.

In [21]:
predictive = MakeMixture(posterior)
predictive.Print()

Blue 171/257
Red 86/257


Now let's consider another scenario.  Suppose I choose a random die and roll it.  If the outcome is red, I tell you the outcome.  Otherwise I choose again and roll again.  Here's a generator that simulates this scenario.

In [22]:
def rollC(dice):
    while True:
        die = dice.Random()
        roll = die.Random()
        if roll == 'Red':
            yield roll

In this scenario, obviously, the outcome is always red:

In [23]:
iterC = rollC(dice)
Pmf(next(iterC) for i in range(1000))

Pmf({'Red': 1.0})

But now suppose I choose and roll until I get a red, and then ask you about the last die I rolled.  What is the probability that it is the reddish die, `d2`?

One each roll, there are four possible results, with these probabilities:

    d1, red       1/2 * 1/3
    d1, blue      1/2 * 2/3
    d2, red       1/2 * 2/3
    d2, blue      1/2 * 1/3
    
On the last roll, I tell you that the outcome is red, so we are left with two possibilities:

    d1, red       1/2 * 1/3
    d2, red       1/2 * 2/3

The likelihood ratio is `2` to `1`, so we can use that to update the prior:

In [24]:
posterior = prior.Copy()
posterior[d1] *= 1
posterior[d2] *= 2
posterior.Normalize()
posterior.Print()

d2 (reddish) 2/3
d1 (bluish)  1/3


And that's the same posterior we saw in Scenarios A and B.  But now the predictive distribution is always red:

In [25]:
predictive = Pmf({'Red':1.0})
predictive.Print()

Red 1.0


Finally, let's consider the scenario where I choose a die once and roll it repeatedly until the out come is red.

In [26]:
def rollD(dice):
    die = dice.Random()
    while True:
        roll = die.Random()
        if roll == 'Red':
            yield roll

Again, obviously, the outcome is always red:

In [27]:
iterD = rollD(dice)
Pmf(next(iterD) for i in range(1000))

Pmf({'Red': 1.0})

But now the probability of getting red is the same regardless of which die I chose.  On average, it takes longer to get to red if I chose `d1`, but if I only tell you the outcome and don't tell you how many times I rolled, you have no way of knowing which die I chose.

So the posterior is the same as the prior.

In [28]:
posterior = prior.Copy()
posterior.Print()

d2 (reddish) 1/2
d1 (bluish)  1/2


And the predictive distribution is, again, always red.

In [29]:
predictive = Pmf({'Red':1.0})

### Summary

In summary, each of the four scenarios yields a different pair of posterior and predictive distributions.

    Scenario        Posterior probability of d2      Predictive probability of red
    A               2/3                              1/2
    B               2/3                              5/9
    C               2/3                              1
    D               1/2                              1

As an aside, Scenario D is interesting if you know how many times I rolled before getting red.  Here's a `Suite` that takes as data the number of times I rolled, including the last one that came up red:

In [84]:
class ScenarioD(Suite):
    def Likelihood(self, data, hypo):
        """
        data: k, number of times I rolled to get a Red
        hypo: a Die object
        """
        p = hypo['Red']
        k = data
        return (1-p)**(k-1) * p

If you know I got red on the first try, that's evidence that I rolled `d2`:

In [85]:
suite = ScenarioD([d1, d2])
suite.Update(1)
suite.Print()

d2 (reddish) 0.6666666666666666
d1 (bluish)  0.3333333333333333


If you know I got it on the second try, that's equally likely with `d1` or `d2`:

In [86]:
suite = ScenarioD([d1, d2])
suite.Update(2)
suite.Print()

d2 (reddish) 0.5
d1 (bluish)  0.5


If it takes three tries or more, that's evidence for `d1`.

In [87]:
suite = ScenarioD([d1, d2])
suite.Update(3)
suite.Print()

d2 (reddish) 0.3333333333333333
d1 (bluish)  0.6666666666666666


So if you know how many times I rolled, you get information about the dice.  But if you don't know how many times I rolled, you don't.

## Medical tests

Suppose we test a patient to see if they have a disease, and the test comes back positive.  What is the probability that the patient is actually sick (that is, has the disease)?

To answer this question, we need to know:

*  The prevalence of the disease in the population the patient is from.  Let's assume the patient is identified as a member of a population where the prevalence is `p`.

*  The sensitivity of the test, `s`, which is the probability of a positive test if the patient is sick.

*  The false positive rate of the test, `t`, which is the probability of a positive test if the patient is not sick.

Given these parameters, we can compute the probability that the patient is sick, given a positive test.

### Test class

To do that, I'll define a `Test` class that extends `Suite`, so it inherits `Update` and provides `Likelihood`.

The instance variables of `Test` are:

*  `p`, `s`, and `t`: Copies of the parameters.
*  `d`: a dictionary that maps from hypotheses to their probabilities.  The hypotheses are the strings `sick` and `notsick`.
*  `likelihood`: a dictionary that encodes the likelihood of the possible data values `pos` and `neg` under the hypotheses.



In [30]:
class Test(Suite):
    """Represents beliefs about a patient based on a medical test."""
    
    def __init__(self, p, s, t):
        # store the parameters
        self.p = p
        self.s = s
        self.t = t
        
        # initialize the prior probabilities
        d = dict(sick=p, notsick=1-p)
        super(Test, self).__init__(d)
        
        # make a nested dictionary to compute likelihoods
        self.likelihood = dict(pos=dict(sick=s, notsick=t),
                               neg=dict(sick=1-s, notsick=1-t))
        
    def Likelihood(self, data, hypo):
        """
        data: 'pos' or 'neg'
        hypo: 'sick' or 'notsick'
        """
        return self.likelihood[data][hypo]

Now we can create a `Test` object with parameters chosen for demonstration purposes (most medical tests are better than this!):

In [31]:
p = 0.1      # prevalence
s = 0.9      # sensitivity
t = 0.3      # false positive rate
p = Fraction(1, 10)
s = Fraction(9, 10)
t = Fraction(3, 10)
test = Test(p, s, t)
test.Print()

notsick 9/10
sick 1/10


If you are curious, here's the nested dictionary that computes the likelihoods:

In [32]:
test.likelihood

{'neg': {'notsick': Fraction(7, 10), 'sick': Fraction(1, 10)},
 'pos': {'notsick': Fraction(3, 10), 'sick': Fraction(9, 10)}}

And here's how we update the `Test` object with a positive outcome:

In [33]:
test.Update('pos')
test.Print()

notsick 3/4
sick 1/4


The positive test provides evidence that the patient is sick, increasing the probability from 0.1 to 0.25.

### Hierarchical model

So far, this is basic Bayesian inference.  Now let's add a wrinkle.  Suppose that we don't know the value of `t` with certainty, but we have reason to believe that `t` is either 0.2 or 0.4 with equal probability.

We can represent this belief with a hierarchical model, where the levels of the hierarchy are:

*  At the top level, the possible values of `t` and their probabilities.
*  At the bottom level, the probability that the patient is sick or not, conditioned on `t`.

To represent the hierarchy, I'll define a `MetaTest`, which is a `Suite` that contains `Test` objects with different values of `t` as hypotheses.

In [34]:
class MetaTest(Suite):
    """Represents a set of tests with different values of `t`."""
    
    def Likelihood(self, data, hypo):
        """
        data: 'pos' or 'neg'
        hypo: Test object
        """
        # the return value from `Update` is the total probability of the
        # data for a hypothetical value of `t`
        return hypo.Update(data)
    
    def Print(self):
        for test, prob in self.Items():
            print('t=', test.t, test, prob)

To update a `MetaTest`, we update each of the hypothetical `Test` objects.  The return value from `Update` is the normalizing constant, which is the total probability of the data under the hypothesis.

We use the normalizing constants from the bottom level of the hierarchy as the likelihoods at the top level.

Here's how we create the `MetaTest` for the scenario we described:

In [35]:
q = 0.5
t1 = 0.2
t2 = 0.4
q = Fraction(1, 2)
t1 = Fraction(2, 10)
t2 = Fraction(4, 10)

test1 = Test(p, s, t1)
test2 = Test(p, s, t2)

metatest = MetaTest({test1:q, test2:1-q})
metatest.Print()

t= 1/5 _nolegend_ 1/2
t= 2/5 _nolegend_ 1/2


At the top level, there are two tests, with different values of `t`.  Initially, they are equally likely.

When we update the `MetaTest`, it updates the embedded `Test` objects and then the `MetaTest` itself.

In [36]:
metatest.Update('pos')

Fraction(9, 25)

Here are the results.

In [37]:
metatest.Print()

t= 1/5 _nolegend_ 3/8
t= 2/5 _nolegend_ 5/8


Because a positive test is more likely if `t=0.4`, the positive test is evidence in favor of the hypothesis that `t=0.4`.

This `MetaTest` object represents what we should believe about `t` after seeing the test, as well as what we should believe about the probability that the patient is sick.

### Predictive probabilities

To compute the probability that the patient is sick, we have to compute the predictive probabilities of `sick` and `notsick`, averaging over the possible values of `t`.  The following function computes this distribution:

In [38]:
def MakeMixture(metapmf, label='mix'):
    """Make a mixture distribution.

    Args:
      metapmf: Pmf that maps from Pmfs to probs.
      label: string label for the new Pmf.

    Returns: Pmf object.
    """
    mix = Pmf(label=label)
    for pmf, p1 in metapmf.Items():
        for x, p2 in pmf.Items():
            mix.Incr(x, p1 * p2)
    return mix

Here's the posterior predictive distribution:

In [39]:
predictive = MakeMixture(metatest)
predictive.Print()

notsick 3/4
sick 1/4


After seeing the test, the probability that the patient is sick is 0.25, which is the same result we got with `t=0.3`.  So at first glance it seems like we might not need the hierarchical model at all.  Maybe we could have computed the mean of `t` and done a simple update.

### Two patients

If the goal is to make a single prediction for a single patient, that's true.  But let's try a different scenario.  Suppose you test two patients and they both test positive.  What is the probability that they are both sick?

To answer that, I define a few more functions to work with Metatests:

In [40]:
def MakeMetaTest(p, s, pmf_t):
    """Makes a MetaTest object with the given parameters.
        
    p: prevalence
    s: sensitivity
    pmf_t: Pmf of possible values for `t`
    """
    tests = {}
    for t, q in pmf_t.Items():
        tests[Test(p, s, t)] = q
    return MetaTest(tests)

def Marginal(metatest):
    """Extracts the marginal distribution of t.
    """
    marginal = Pmf()
    for test, prob in metatest.Items():
        marginal[test.t] = prob
    return marginal

def Conditional(metatest, t):
    """Extracts the distribution of sick conditioned on t."""
    for test, prob in metatest.Items():
        if test.t == t:
            return test

`MakeMetaTest` makes a `MetaTest` object starting with a given PMF of `t`.

`Marginal` extracts the PMF of `t` from a `MetaTest`.

`Conditional` takes a specified value for `t` and returns the PMF of `sick` and `notsick` conditioned on `t`.

I'll test it out using the same parameters from above:

In [41]:
p = Fraction(1, 10)
s = Fraction(9, 10)
t = Fraction(3, 10)
q = Fraction(1, 2)
t1 = Fraction(2, 10)
t2 = Fraction(4, 10)

pmf_t = Pmf({t1:q, t2:1-q})
pmf_t.Print()

1/5 1/2
2/5 1/2


Here are the results

In [42]:
metatest = MakeMetaTest(p, s, pmf_t)
metatest.Update('pos')
metatest.Print()

t= 1/5 _nolegend_ 3/8
t= 2/5 _nolegend_ 5/8


Same as before.  Now we can extract the posterior distribution of `t`.

In [43]:
Marginal(metatest).Print()

1/5 3/8
2/5 5/8


Having seen one positive test, we are a little more inclined to believe that `t=0.4`; that is, that the false positive rate is high.

And we can extract the conditional distributions for the patient:

In [44]:
cond1 = Conditional(metatest, t1)
cond1.Print()

notsick 2/3
sick 1/3


In [45]:
cond2 = Conditional(metatest, t2)
cond2.Print()

notsick 4/5
sick 1/5


Finally, we can make a predictive distribution for the patient, which is a weighted mixture of the conditional distributions:

In [46]:
predictive = MakeMixture(metatest)
predictive.Print()

notsick 3/4
sick 1/4


At this point we have a `MetaTest` that contains our updated information about the test (the distribution of `t`) and about the patient that tested positive.

Now, to compute the probability that both patients are sick, we have to know the distribution of `t` for both patients.  And that depends on details of the scenario.  There are two possibilities:

*  Scenario A: Suppose there are two different tests, with different false positive rates, and we don't know which tests were used.  Or suppose that the false positive rate is different for different groups of people, and we don't know which group the patients are from.  In either case, the values of `t` for the two patients are independent.

*  Scenario B: Alternatively, suppose there is only one test and we have reason to believe that the false positive rate is the same for everyone, so `t` is either `t1` for everyone or `t2` for everyone, but we don't know which.  In that case, the value of `t` for the two patients is the same.

The probability that both patients are sick is different in these two scenarios.  I'll compute both.

### Scenario A

I'll start by simulating the scenario, which helps specify the scenario unambiguously, and also tells us what the answer should be, at least approximately:

In [47]:
from random import random

def flip(p):
    return random() < p

In [48]:
def generate_pair_A(p, s, pmf_t):
    while True:
        sick1, sick2 = flip(p), flip(p)
        
        t = pmf_t.Random()
        test1 = flip(s) if sick1 else flip(t)

        t = pmf_t.Random()
        test2 = flip(s) if sick2 else flip(t)

        if test1 and test2:
            yield sick1, sick2

In [49]:
p = 0.1
s = 0.9
q = 0.5
t1 = 0.2
t2 = 0.4
pmf_t = Pmf({t1:q, t2:1-q})

pair_iterator = generate_pair_A(p, s, pmf_t)
outcomes = Pmf(next(pair_iterator) for i in range(100000))
outcomes.Print()

(False, False) 0.5621400000000001
(False, True) 0.18672000000000002
(True, False) 0.18876
(True, True) 0.062380000000000005


In Scenario A, the probability that either patient is sick is independent of the other, the values of `t` are independent, and the probability that either patient tests positive is independent of the other.

So we can generate predictive distributions for the two patients separately and then compute the "sum" of those distributions (which is their convolution):

In [50]:
conjunction = MakeMixture(metatest) + MakeMixture(metatest)
conjunction.Print()

notsicknotsick 9/16
notsicksick 3/16
sicknotsick 3/16
sicksick 1/16


So the probability both patients are sick is 1/16 = 0.0625.

As an aside, it is equivalent to convolve the two distributions first and then compute the predictive distribution, but the first way is more efficient.

In [51]:
conjunction = MakeMixture(metatest+metatest)
conjunction.Print()

notsicknotsick 9/16
notsicksick 3/16
sicknotsick 3/16
sicksick 1/16


Either way, the results are consistent with the simulation.

### Scenario B

In Scenario B we have to work a little harder, because the two patients are not independent of each other.  Each patient gives us information about the test, which influences how we intererpret the test for the patients.

And, when we generate the predictive distribution, we have to account for the dependence of the values of `t`.

Again, I'll start with a simulation:

In [52]:
def generate_pair_B(p, s, pmf_t):
    while True:
        sick1, sick2 = flip(p), flip(p)
        
        t = pmf_t.Random()
        test1 = flip(s) if sick1 else flip(t)

        #  Here's the difference
        #  t = pmf_t.Random()
        test2 = flip(s) if sick2 else flip(t)

        if test1 and test2:
            yield sick1, sick2

The difference between Scenario A and Scenario B is the line I commented out.  In Scenario B, we generate `t` once and it applies to both patients.

In [53]:
p = 0.1
s = 0.9
q = 0.5
t1 = 0.2
t2 = 0.4
pmf_t = Pmf({t1:q, t2:1-q})

pair_iterator = generate_pair_B(p, s, pmf_t)
outcomes = Pmf(next(pair_iterator) for i in range(100000))
outcomes.Print()

(False, False) 0.5883700000000001
(False, True) 0.17602
(True, False) 0.17540000000000003
(True, True) 0.06021000000000001


I'll start again with a `Metatest` that represents posterior belief about the test and the first patient.

In [54]:
p = Fraction(1, 10)
s = Fraction(9, 10)
t = Fraction(3, 10)
q = Fraction(1, 2)
t1 = Fraction(2, 10)
t2 = Fraction(4, 10)
pmf_t = Pmf({t1:q, t2:1-q})

metatest1 = MakeMetaTest(p, s, pmf_t)
metatest1.Update('pos')
metatest1.Print()

t= 1/5 _nolegend_ 3/8
t= 2/5 _nolegend_ 5/8


Now suppose the second patient arrives.  We need a new `MetaTest` that contains the updated information about the test, but no information about the patient other than the prior proability of being sick, `p`:

In [55]:
metatest2 = MakeMetaTest(p, s, Marginal(metatest1))
metatest2.Print()

t= 1/5 _nolegend_ 3/8
t= 2/5 _nolegend_ 5/8


Now we can update this `MetaTest` with the result from the second test:

In [56]:
metatest2.Update('pos')
metatest2.Print()

t= 1/5 _nolegend_ 9/34
t= 2/5 _nolegend_ 25/34


This distribution contains updated information about the test, based on two positive outcomes, and updated information about a patient who has tested positive (once).

In [57]:
predictive = MakeMixture(metatest2)
predictive.Print()

notsick 13/17
sick 4/17


After two tests, the probability that the patient is sick is slightly lower than after one (4/17 is about 0.235, compared to 0.25).  That's because the second positive test increases our belief that the false positive rate is high (t=0.4), which decreases our belief that either patient is sick.

To compute the probability that both are sick, we can't just convolve the predictive distribution with itself, because the selection of `t` is not independent for the two patients.  Instead, we have to make a weighted mixture of conditional distributions.

If we know `t=t1`, we can compute the joint distribution for the two patients:

In [58]:
cond_t1 = Conditional(metatest2, t1)
conjunction_t1 = cond_t1 + cond_t1
conjunction_t1.Print()

notsicknotsick 4/9
notsicksick 2/9
sicknotsick 2/9
sicksick 1/9


If we know that `t=t1`, the probability of `sicksick` is 0.111.  And for `t=t2`:

In [59]:
cond_t2 = Conditional(metatest2, t2)
conjunction_t2 = cond_t2 + cond_t2
conjunction_t2.Print()

notsicknotsick 16/25
notsicksick 4/25
sicknotsick 4/25
sicksick 1/25


If we know that `t=t2`, the probability of `sicksick` is 0.04.

The overall probability of `sicksick` is the weighted average of these probabilities:

In [60]:
posterior_t = Marginal(metatest2)
posterior_t[t1] * conjunction_t1['sicksick'] + posterior_t[t2] * conjunction_t2['sicksick']

Fraction(1, 17)

1/17 is about 0.0588.

To compute the probabilities for all four outcomes, I'll make a `Metapmf` that contains the two conditional distributions.

In [61]:
metapmf = Pmf()
for t, prob in Marginal(metatest2).Items():
    cond = Conditional(metatest2, t)
    conjunction = cond + cond
    metapmf[conjunction] = prob
    
metapmf.Print()

_nolegend_ 25/34
_nolegend_ 9/34


And finally we can use `MakeMixture` to compute the weighted averages of the posterior probabilities:

In [62]:
predictive = MakeMixture(metapmf)
predictive.Print()

notsicknotsick 10/17
notsicksick 3/17
sicknotsick 3/17
sicksick 1/17


### Summary

In summary:

*  In Scenario A, the values of `t` for the two patients are independent, and the probability that both are sick is 1/16 = 0.0625.

*  In Scenario B, the value of `t` has to be the same for both patients, and the probability that both are sick is 1/17 ≈ 0.0588.

A real scenario might combine elements of both; that is, the false positive rate might be different for different people, and we might have some uncertainty about what it is.  In that case, the most accurate predictive probability might be anywhere between the values we computed.

### Scenarios C and D

In Scenario B, I assumed that we see all patients regardless of whether they are sick or not, test positive or not.

In that case, when we see a positive test, it provides evidence that the false positive rate is high.  As a result, as we see more patients, we get more and more confident about the value of `t`.

I'll demonstrate this with a simulation.  Here are the usual parameters:

In [63]:
p = 0.1
s = 0.9
q = 0.5
t1 = 0.2
t2 = 0.4
pmf_t = Pmf({t1:q, t2:1-q})

And here's a generator that simulates patients for given parameters:

In [64]:
def generate_patient_all(p, s, t):
    while True:
        sick = flip(p)
        test = flip(s) if sick else flip(t)
        yield 'pos' if test else 'neg'

Now we can simulate a doctor who sees 100 patients and updates `metatest` each time.

In [65]:
def run_simulation(p, s, pmf_t, iterator):
    metatest = MakeMetaTest(p, s, pmf_t)

    for i in range(100):
        data = next(iterator)
        metatest = MakeMetaTest(p, s, Marginal(metatest))
        metatest.Update(data)

    Marginal(metatest).Print()

If `t` is actually 0.2, the doctor eventually becomes convinced that `t=0.2`

In [66]:
t = 0.2
iterator = generate_patient_all(p, s, t)
run_simulation(p, s, pmf_t, iterator)

0.2 0.9949736445038064
0.4 0.005026355496193581


And if `t` is actually 0.4, the doctor eventually becomes convinced that `t=0.4`

In [67]:
t = 0.4
iterator = generate_patient_all(p, s, t)
run_simulation(p, s, pmf_t, iterator)

0.2 0.000601664342045809
0.4 0.9993983356579543


So far, so good.  

But what if the doctor is a specialist who only sees patients after they have tested positive?

Here's a generator that simulates this scenario.

In [68]:
def generate_patient_posonly(p, s, t):
    while True:
        sick = flip(p)
        test = flip(s) if sick else flip(t)
        if test:
            yield 'pos'

Now if the doctor applies the same logic as before, updating their belief about the test each time they see a positive test, they are quickly convinced that `t` is high, regardless of the actual value

In [69]:
t = 0.2
iterator = generate_patient_posonly(p, s, t)
run_simulation(p, s, pmf_t, iterator)

0.2 6.533186235000651e-23
0.4 1.0


In [70]:
t = 0.4
iterator = generate_patient_posonly(p, s, t)
run_simulation(p, s, pmf_t, iterator)

0.2 6.533186235000651e-23
0.4 1.0


So that's not good.

We have to figure out how to update our belief about `t` in this case.  I'll define `r` as the referral rate for patients who test negative.  If `r=1`, we see all patients, as in Scenarios A and B.

If `r=0` we only see patients who tests positive.

If we know `p`, `s`, `t`, and `r`, we can compute the probability of seeing a patient with a positive test:

In [71]:
def prob_pos(p, s, t, r):
    yes = p*s + (1-p) * t
    no = p * (1-s) + (1-p) * (1-t)
    return yes / (yes + no * r)

Here are the probabilities of seeing a patient with a positive test for the two values of `t`:

In [72]:
p = Fraction(1, 10)
s = Fraction(9, 10)
t = Fraction(3, 10)
q = Fraction(1, 2)
t1 = Fraction(2, 10)
t2 = Fraction(4, 10)
pmf_t = Pmf({t1:q, t2:1-q})

pp1 = prob_pos(p, s, t1, 1)
pp2 = prob_pos(p, s, t2, 1)
pp1, pp2

(Fraction(27, 100), Fraction(9, 20))

Since these probabilities are the likelihood of the data, we can use them to update our belief about `t`.  Here's what we get with `r=1`.

In [73]:
pmf_t = Pmf({t1:q, t2:1-q})
pmf_t[t1] *= prob_pos(p, s, t1, r=1)
pmf_t[t2] *= prob_pos(p, s, t2, r=1)
pmf_t.Normalize()
pmf_t.Print()

1/5 3/8
2/5 5/8


And that's consistent with what we saw in Scenarios A and B.

But when `r=0`, we only see patients with positive test.  The probability of the data is 1, regardless of `t`, so the data have no effect on our belief about `t`.

In [74]:
pmf_t = Pmf({t1:q, t2:1-q})
pmf_t[t1] *= prob_pos(p, s, t1, 0)
pmf_t[t2] *= prob_pos(p, s, t2, 0)
pmf_t.Normalize()
pmf_t.Print()

1/5 1/2
2/5 1/2


To compute the probability that the patient is sick, we can make a `MetaTest` and update the `Test` objects it contains, but we don't update the top level of the hierarchy.

In [75]:
metatest = MakeMetaTest(p, s, pmf_t)
for test in metatest:
    test.Update('pos')
    
metatest.Print()

t= 1/5 _nolegend_ 1/2
t= 2/5 _nolegend_ 1/2


Now we can generate the predictive distribution as usual:

In [76]:
predictive = MakeMixture(metatest)
predictive.Print()

notsick 11/15
sick 4/15


To compute the probability that two patients who test positive are sick, we have to deal with two cases again.  

### Scenario C

If the value of `t` is independent for all patients, we just compute the convolution of the predictive distribution with itself.

In [77]:
conjunction = predictive + predictive
conjunction.Print()

notsicknotsick 121/225
notsicksick 44/225
sicknotsick 44/225
sicksick 16/225


### Scenario D

Or, if we think the value of `t` is the same for all patients, we have to use the same technique we used in Scenario B.

In [78]:
metapmf = Pmf()
for t, prob in Marginal(metatest).Items():
    cond = Conditional(metatest, t)
    conjunction = cond + cond
    metapmf[conjunction] = prob
    
metapmf.Print()

_nolegend_ 1/2
_nolegend_ 1/2


In [79]:
MakeMixture(metapmf).Print()

notsicknotsick 122/225
notsicksick 43/225
sicknotsick 43/225
sicksick 17/225


In [80]:
16/255, 17/225

(0.06274509803921569, 0.07555555555555556)

### Summary

There are four possible answers to this question, depending on two details of the scenario.

Scenario A: We see all patients, and different patients/tests have different values of `t`.  In this case, the probability one patient is sick, given a positive test, is 1/4, and the probability that two patients are sick, given that each of them has tested positive, is 1/16 = 0.0625.

Scenario B: We see all patients, and the value of `t` is the same for everyone.  The probability that one patient is sick is 1/4 (same as A), but the probability for two patients is 4/17 = 0.0588.

Scenario C: We only see patients who test positive, and different patients/tests have different values of `t`.  The probability for one patient is 4/15; for two it's 16/225.

Scenario D: We only see patients who test positive, and the value of `t` is the same for everyone.  The probability for one patient is 4/15 (same as C); for two it's 17/225.